# This lab contains exercises for the Bioinformatics with Python Lab held on November 19th, 2024 during the CSHL Advanced Sequencing Technologies & Bioinformatics Analysis course

## Using `requests` to extract data from an API

#### First, let's import the `requests` library

In [ ]:
import requests

#### We are interested in using `requests` to query the Ensembl API to get back a record for a gene of interest. Run the cell below to see how this can be done:

In [ ]:
gene_symbol = "TCERG1" # Replace with gene symbol in between the quotations
ensembl_gene_url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene_symbol}?content-type=application/json"
ensembl_gene_record = requests.get(ensembl_gene_url)

#### 1. What do you see when you run the cell below?

In [ ]:
ensembl_gene_record

*Type Answer Here*:

#### 2. What do you see when you run the cell below? How is this different from the output above?

In [ ]:
ensembl_gene_record.json()

*Type Answer Here*:

## Introduction to Web Scraping using `beautifulsoup4` 

#### We're now going to be using `beautifulsoup4` to practice web scraping from the course website: https://meetings.cshl.edu/courses.aspx?course=C-SEQTEC

In [ ]:
from bs4 import BeautifulSoup

#### 1. Run the code block below to get the list of invited speakers for the course. What does the output look like?

In [ ]:
url = 'https://meetings.cshl.edu/courses.aspx?course=C-SEQTEC'
response = requests.get(url)
html_content = response.text
cshl_webpage = BeautifulSoup(html_content, "html.parser")
cshl_webpage.find('div', class_='cspeakers16').find("div", class_="cspeakers16")

*Type Answer Here*: 

#### The code below can be run to convert this output to a human-readable form:

In [ ]:
instructors = cshl_webpage.find('div', class_='cspeakers16').find("div", class_="cspeakers16")
insructors = instructors.get_text().replace("\xa0", " ").strip()
print(insructors)

#### 2. Suppose we want to extract the dates for the course, and we know that the dates are under the `cdate16` flag. Write a query to output the dates that uses the `get_text()` function

In [ ]:
# Uncomment and type code here

## Storing and Accessing Data using Environment Variables

## VCFs and Pandas

#### We will conclude by doing some exercises that combine analysis of variant call files (VCFs) and the pandas library. First, let's import pandas and pysam:

In [ ]:
import pandas as pd
import pysam

#### We would like to read in our VCF file using pysam (if you have an error statement at the bottom of your output, please ignore):

In [ ]:
vcf_file = pysam.VariantFile("../data/Exome_Norm_HC_calls.filtered.vcf.gz", index_filename=None)
print(vcf_file.header)

#### 1. Scroll down to the bottom of the output and look at the rows with the ##FORMAT flags. What do these rows mean and why are they important?

*Type Answer Here*:

#### 2. Run the code below to view the first 30 variants in the vcf. What kind of variants do you see in this list? How can you tell?

In [ ]:
print("#CHROM POS ID REF ALT QUAL FILTER INFO FORMAT HG001")
for index, record in enumerate(vcf_file):
   if index == 30:
       break
   else:
       print(record)

*Type Answer Here*:

#### Run the cell below to convert the VCF data to a pandas data frame for downstream processing. This should take around 30 seconds to run.

In [ ]:
columns = ["chrom", "pos", "id", "ref", "alt", "qual", "filter",
          "dp", "gt", "ad", "gq"]
vcf_data = []
for record in vcf_file:
   sample_data = record.samples["HG001"]
   vcf_data.append({"chrom": record.chrom,
                  "pos": record.pos,
                  "id": record.id,
                  "ref": record.ref,
                  "alt": ','.join(record.alts),
                  "qual": record.qual,
                  "filter": ';'.join(record.filter.keys()),
                  "dp": sample_data.get("DP"),
                  "gt": sample_data.get("GT"),
                  "ad": sample_data.get("AD"),
                  "gq":sample_data.get("GQ")})
vcf_data = pd.DataFrame(vcf_data)
vcf_data["chrom"] = vcf_data["chrom"].str.replace('chr', '', regex=False).astype(int)

#### 3. How many rows are in the dataframe? What pandas command can you use to determine this? 

#### 4. We would like to take a random sample of 20 entries from the dataframe. How would we do this in pandas? Name the new dataset `vcf_subset`. Set the random state to equal 7 for reproducibility.

#### 5. Suppose we want to sort the sample so that both the chromosome numbers are in **ascending order** and the variant positions are in **descending order**. How would you do this using pandas?

#### 6. We would like to only include variants that have a read depth >= 500.0 and a genotype quality >= 400.0. How would you do this using pandas?

## Final Exercise

#### A clinical colleague has asked you to examine the VCF to see if there are any frameshift variants in the coding regions of the *CELA1* gene. How many frameshift variants are there? Write your code below to answer this question. The exon coordinates for the *CELA1* gene are provided in the cell below.

In [ ]:
exon_list = [[51346622,51346679], [51345794,51345877], [51343752,51343853],
             [51342574,51342700], [51341243,51341380], [51339859,51340005],
             [51329683,51329833], [51328441,51328594]]